In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
%matplotlib inline

In [27]:
# pandas global settings
pd.options.display.max_columns=500
pd.options.display.max_rows=500

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [28]:
path="../../../../Datafiles/LendingClub/"
file = 'LoanStats3d2015.csv.zip'
url = 'https://resources.lendingclub.com/LoanStats3d.csv'

print("We are working with 2015 Lending dataset from {}".format(url))
y2015 = pd.read_csv(path + file, header=1, skipfooter=4, sep=',', quotechar='"', engine='python', compression='zip')
print("there are {:,} entries in the data frame, with {} columns, from data file: {}".format(len(y2015),
                                                                                             len(y2015.columns),                                                                                            file))
y2015.head(20)
print("y2015 has {} rows.".format(len(y2015)))

We are working with 2015 Lending dataset from https://resources.lendingclub.com/LoanStats3d.csv
there are 421,095 entries in the data frame, with 144 columns, from data file: LoanStats3d2015.csv.zip
y2015 has 421095 rows.


In [29]:
y2015.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,24700,24700,24700.0,36 months,11.99%,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,NaN,NaN,small_business,Business,577xx,SD,16.06,1,Dec-1999,4,6.0,NaN,22,0,21470,19.2%,38,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Apr-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,204396,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800,0.0,0.0,6.0,4,9733,57830.0,27.1,0,0,113.0,192,2,2,4,2.0,NaN,0.0,6.0,0,5,5,13,17,6,20,27.0,5,22,0.0,0,0,2,97.4,7.7,0,0,314017,39475,79300,24667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,35000,35000,35000.0,60 months,14.85%,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,076xx,NJ,17.06,0,Sep-2008,0,NaN,NaN,13,0,7802,11.6%,17,w,14623.48,14623.48,33123.810000,33123.81,20376.52,12747.29,0.0,0.0,0.0,Apr-2019,829.90,May-2019,Apr-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,301500,1.0,1.0,0.0,1.0,23.0,12609.0,70.0,1.0,1.0,6987.0,45.0,67300,0.0,1.0,0.0,2,23192,54962.0,12.1,0,0,36.0,87,2,2,1,2.0,NaN,NaN,NaN,0,4,5,8,10,2,10,13.0,5,13,0.0,0,0,1,100.0,0.0,0,0,381215,52226,62500,18000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3600,3600,3600.0,36 months,13.99%,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0,Aug-2003,1,30.0,NaN,7,0,2765,29.7%,13,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN

In [30]:
y2015.tail(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
421090,NaN,NaN,10000,10000,10000.0,36 months,11.99%,332.10,B,B5,Office Assistant,8 years,RENT,31000.0,Verified,Jan-2015,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,905xx,CA,28.69,0,Sep-2004,0,NaN,77.0,9,1,14037,82.1%,15,f,0.0,0.0,11947.244045,11947.24,10000.00,1947.24,0.0,0.00,0.0000,Jan-2018,331.90,NaN,Feb-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,25274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17100,NaN,NaN,NaN,2,3159,448.0,95.1,0,0,122.0,115,17,10,0,90.0,NaN,9.0,NaN,0,3,6,3,4,7,7,8.0,6,9,0.0,0,0,1,100.0,100.0,0,0,32950,25274,9200,15850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421091,NaN,NaN,24000,24000,24000.0,36 months,11.99%,797.03,B,B5,Director of Maintenance,10+ years,MORTGAGE,79000.0,Verified,Jan-2015,Fully Paid,n,NaN,NaN,home_improvement,Home improvement,072xx,NJ,3.90,0,Mar-1974,1,26.0,NaN,5,0,8621,84.5%,23,f,0.0,0.0,28663.686660,28663.69,24000.00,4663.69,0.0,0.00,0.0000,Feb-2018,2.58,NaN,Jan-2019,0,29.0,1,Individual,NaN,NaN,NaN,0,0,140285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10200,NaN,NaN,NaN,3,35071,976.0,89.2,0,0,NaN,488,3,3,4,3.0,29.0,3.0,29.0,10,2,3,3,14,0,4,17.0,3,5,0.0,0,0,2,56.5,100.0,0,0,152650,8621,9000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421092,NaN,NaN,13000,13000,13000.0,60 months,15.99%,316.07,D,D2,Radiologist Technologist,5 years,RENT,35000.0,Verified,Jan-2015,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,378xx,TN,30.90,0,Sep-2003,0,NaN,84.0,9,1,11031,61.3%,22,w,0.0,0.0,6109.520000,6109.52,2527.70,2506.32,0.0,1075.50,193.5900,May-2016,316.07,NaN,Dec-20

In [31]:
for col in y2015.columns:
    print("column: {}".format(col))

column: id
column: member_id
column: loan_amnt
column: funded_amnt
column: funded_amnt_inv
column: term
column: int_rate
column: installment
column: grade
column: sub_grade
column: emp_title
column: emp_length
column: home_ownership
column: annual_inc
column: verification_status
column: issue_d
column: loan_status
column: pymnt_plan
column: url
column: desc
column: purpose
column: title
column: zip_code
column: addr_state
column: dti
column: delinq_2yrs
column: earliest_cr_line
column: inq_last_6mths
column: mths_since_last_delinq
column: mths_since_last_record
column: open_acc
column: pub_rec
column: revol_bal
column: revol_util
column: total_acc
column: initial_list_status
column: out_prncp
column: out_prncp_inv
column: total_pymnt
column: total_pymnt_inv
column: total_rec_prncp
column: total_rec_int
column: total_rec_late_fee
column: recoveries
column: collection_recovery_fee
column: last_pymnt_d
column: last_pymnt_amnt
column: next_pymnt_d
column: last_credit_pull_d
column: collect

## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas to generate a dummy variable for every categorical colummn and see what happens off of this kind of naive approach.

In [32]:
# from sklearn import ensemble
# from sklearn.model_selection import cross_val_score

# rfc = ensemble.RandomForestClassifier()
# X = y2015.drop('loan_status', 1)
# Y = y2015['loan_status']
# X = pd.get_dummies(X)

# cross_val_score(rfc, X, Y, cv=5)

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [33]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

term
2
int_rate
111
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
2
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
52
next_pymnt_d
7
last_credit_pull_d
53
application_type
2
verification_status_joint
1
hardship_flag
2
hardship_type
1
hardship_reason
9
hardship_status
3
hardship_start_date
27
hardship_end_date
28
payment_plan_start_date
28
hardship_loan_status
4
debt_settlement_flag
2
debt_settlement_flag_date
45
settlement_status
3
settlement_date
48


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [34]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [35]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
421090,NaN,NaN,10000,10000,10000.0,36 months,11.99,332.10,B,8 years,RENT,31000.0,Verified,Jan-2015,Fully Paid,n,debt_consolidation,Debt consolidation,28.69,0,0,NaN,77.0,9,1,14037,15,f,0.0,0.0,11947.244045,11947.24,10000.00,1947.24,0.0,0.00,0.0000,Jan-2018,331.90,NaN,Feb-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,25274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17100,NaN,NaN,NaN,2,3159,448.0,95.1,0,0,122.0,115,17,10,0,90.0,NaN,9.0,NaN,0,3,6,3,4,7,7,8.0,6,9,0.0,0,0,1,100.0,100.0,0,0,32950,25274,9200,15850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421091,NaN,NaN,24000,24000,24000.0,36 months,11.99,797.03,B,10+ years,MORTGAGE,79000.0,Verified,Jan-2015,Fully Paid,n,home_improvement,Home improvement,3.90,0,1,26.0,NaN,5,0,8621,23,f,0.0,0.0,28663.686660,28663.69,24000.00,4663.69,0.0,0.00,0.0000,Feb-2018,2.58,NaN,Jan-2019,0,29.0,1,Individual,NaN,NaN,NaN,0,0,140285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10200,NaN,NaN,NaN,3,35071,976.0,89.2,0,0,NaN,488,3,3,4,3.0,29.0,3.0,29.0,10,2,3,3,14,0,4,17.0,3,5,0.0,0,0,2,56.5,100.0,0,0,152650,8621,9000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421092,NaN,NaN,13000,13000,13000.0,60 months,15.99,316.07,D,5 years,RENT,35000.0,Verified,Jan-2015,Charged Off,n,debt_consolidation,Debt consolidation,30.90,0,0,NaN,84.0,9,1,11031,22,w,0.0,0.0,6109.520000,6109.52,2527.70,2506.32,0.0,1075.50,193.5900,May-2016,316.07,NaN,Dec-2016,0,NaN,1,Individual,NaN,NaN,NaN,0,0,34178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18000,NaN,NaN,NaN,7,3798,2554.0,75.9,0,0,134.0,108,6,6,0,17.0,NaN,10.0,NaN,0,4,6,4,4,12,7,10.0,6,9,0.0,0,0,3,100.0,50.0,1,0,51239,34178,10600,33239,NaN,NaN,NaN,NaN

In [36]:
# Remove two summary rows at the end that don't actually contain data.
# y2015 = y2015[:-2]
# skipfooter=4 in read_csv fixed this better...

Now this should be better. Let's try again.

In [37]:
y2015.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,24700,24700,24700.0,36 months,11.99,820.28,C,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,small_business,Business,16.06,1,4,6.0,NaN,22,0,21470,38,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Apr-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,204396,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800,0.0,0.0,6.0,4,9733,57830.0,27.1,0,0,113.0,192,2,2,4,2.0,NaN,0.0,6.0,0,5,5,13,17,6,20,27.0,5,22,0.0,0,0,2,97.4,7.7,0,0,314017,39475,79300,24667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,35000,35000,35000.0,60 months,14.85,829.90,C,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,n,debt_consolidation,Debt consolidation,17.06,0,0,NaN,NaN,13,0,7802,17,w,14623.48,14623.48,33123.810000,33123.81,20376.52,12747.29,0.0,0.0,0.0,Apr-2019,829.90,May-2019,Apr-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,301500,1.0,1.0,0.0,1.0,23.0,12609.0,70.0,1.0,1.0,6987.0,45.0,67300,0.0,1.0,0.0,2,23192,54962.0,12.1,0,0,36.0,87,2,2,1,2.0,NaN,NaN,NaN,0,4,5,8,10,2,10,13.0,5,13,0.0,0,0,1,100.0,0.0,0,0,381215,52226,62500,18000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3600,3600,3600.0,36 months,13.99,123.03,C,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,debt_consolidation,Debt consolidation,5.91,0,1,30.0,NaN,7,0,2765,13,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Apr-2019,0,30.0,1,Individual,NaN,NaN,NaN,0,722,144904,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300,3.0,1.0,4.0,4,20701,1506.0,37.2,0,0,148.0,128,3,3,1,4.0,69.0,4.0,69.0,2,2,4,2,5,3,4,9.0,4,7,0.0,0,0,3,76.9,0.0,0,0,178

It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [38]:
print("this is the first real attempt, the first one croaked.\nthis takes about 4 minutes to run, just so you know, on my Mac.")
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
Y = y2015['loan_status']
X = y2015.drop('loan_status', 1)
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

this is the first real attempt, the first one croaked.
this takes about 4 minutes to run, just so you know, on my Mac.


/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Use

array([0.9896469 , 0.9909527 , 0.99197359, 0.99209214, 0.99301829,
       0.9932556 , 0.99349292, 0.99320794, 0.99261423, 0.99320762])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [39]:
# My code below.

#### We need to determine which features contribute statistically to this model, so here we are going to determine the feature importance

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)
## Fit the model on your training data.
rfc.fit(X_train, y_train) 
## And score it on your testing data.
rfc.score(X_test, y_test)

/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9919554969781166

In [41]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [42]:
feature_importances.head(36)*100

,importance
recoveries,15.698711
out_prncp,8.972966
collection_recovery_fee,7.993030
total_rec_prncp,7.759095
next_pymnt_d_May-2019,7.260468
out_prncp_inv,6.894576
total_pymnt_inv,5.514792
last_pymnt_d_Apr-2019,3.589984
term_ 60 months,3.535020
term_ 36 months,3.247253


In [43]:
# feature_importances.stack().index[np.argmax(feature_importances.values)]
feature_importances.to_csv(path + 'feature_importances.csv')

In [44]:
### After feature importance columns determined

In [45]:
rfc = ensemble.RandomForestClassifier()
Y = y2015['loan_status']
X = y2015.drop('loan_status', 1)
X = pd.get_dummies(X)
X = X.dropna(axis=1)

# X = X[['out_prncp','collection_recovery_fee','recoveries','total_rec_prncp','next_pymnt_d_May-2019',
#        'total_pymnt','out_prncp_inv','total_pymnt_inv','total_rec_int','last_pymnt_amnt',
#        'term_ 60 months','funded_amnt','funded_amnt_inv','last_pymnt_d_Apr-2019','loan_amnt',
#        'installment','debt_settlement_flag_Y','settlement_status_COMPLETE','term_ 36 months',
#        'last_credit_pull_d_Apr-2019','int_rate','debt_settlement_flag_N','last_credit_pull_d_Feb-2017',
#        'total_rec_late_fee','settlement_status_ACTIVE','last_pymnt_d_Mar-2019','last_credit_pull_d_Oct-2016',
#        'grade_A','revol_bal','annual_inc','avg_cur_bal','tot_cur_bal','mo_sin_old_rev_tl_op',
#        'tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit']] # 99%

# X = X[['out_prncp','collection_recovery_fee','recoveries','total_rec_prncp','next_pymnt_d_May-2019',
#        'total_pymnt','out_prncp_inv','total_pymnt_inv','total_rec_int','last_pymnt_amnt',
#        'term_ 60 months','funded_amnt','funded_amnt_inv','last_pymnt_d_Apr-2019','loan_amnt',
#        'installment','debt_settlement_flag_Y','settlement_status_COMPLETE','term_ 36 months',
#        'last_credit_pull_d_Apr-2019','int_rate']] # 99% still!
# X = X[['out_prncp','collection_recovery_fee','recoveries','total_rec_prncp','next_pymnt_d_May-2019',
#        'total_pymnt','out_prncp_inv','total_pymnt_inv','total_rec_int','last_pymnt_amnt',
#        'term_ 60 months','funded_amnt']] # 98% still!
# X = X[['out_prncp','collection_recovery_fee','recoveries','total_rec_prncp','next_pymnt_d_May-2019',
#        'total_pymnt','out_prncp_inv']] # 97% still!
# X = X[['out_prncp']] # 81%
# X = X[['out_prncp','collection_recovery_fee','recoveries','total_rec_prncp']] # 98% 
# X = X[['out_prncp','collection_recovery_fee']] # 93%!!! - 2 features predicting 93% of the model 

X = X[['recoveries','total_rec_int','term_ 60 months','funded_amnt','funded_amnt_inv','loan_amnt',
       'term_ 36 months','last_credit_pull_d_Apr-2019','int_rate','last_credit_pull_d_Feb-2017',
       'total_rec_late_fee','last_credit_pull_d_Oct-2016','grade_A','revol_bal','annual_inc','tot_cur_bal',
       'tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit']] # 92% -- no payment amount, or outstanding principal

cross_val_score(rfc, X, Y, cv=10)

/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Use

array([0.92508252, 0.9253182 , 0.93087317, 0.92861553, 0.92861553,
       0.93065615, 0.93124822, 0.92816092, 0.92588107, 0.92704128])